In [625]:
import time
import datetime
import requests
from bs4 import BeautifulSoup
import smtplib
from email.mime.text import MIMEText
from time import gmtime, strftime
import csv
import re
import numpy as np


username = ''
password = ''
login_url = 'http://www.neopets.com/login.phtml'
with requests.Session() as s:
    s.get(login_url)
    payload = {'destination': '', 'username': username, 'password': password}
    print('Logging in to the site.')
    r = s.post(login_url, data=payload)
    r = s.get('http://www.neopets.com/medieval/shapeshifter.phtml')
    soup = BeautifulSoup(r.content, 'html.parser')

activeShape = soup.find( height = '50')
nextShape = activeShape.next_sibling.table
activeShape = activeShape.table

Java = soup.find(language="JavaScript1.2")
r = Java.get_text().find('gX = ') 
s = Java.get_text().find('gY = ') 
print 
gX = int(Java.get_text()[r + 5])
gY = int(Java.get_text()[s + 5])

matrix = np.zeros(gX*gY).reshape(gY,gX)
for i in range(gY):
    for j in range(gX):
        loc = Java.get_text().find('imgLocStr['+str(j)+']['+str(i) + ']')
        if Java.get_text()[loc+19:loc+22].encode('ascii','ignore') == "swo":
            matrix[i,j] = 1





Logging in to the site.



In [626]:
GivenShape = []
for k in range(len(nextShape.find_all(cellspacing = "0")) + 1):
    if k == 0:
        q = activeShape
    elif k != 0:
        q = nextShape.find_all(cellspacing = "0")[k - 1]
    ii = len(q.find_all('tr'))
    jj = (len(q.find_all('td')) )/ii
    a = np.zeros(ii*jj).reshape(ii,jj)
    for i in range(ii):
        for j in range(jj):
            a[i][j] = len(q.find_all('td')[jj*i + j] )
    GivenShape.append(a)

array = []
for x in GivenShape:
    b = []
    for j in range(gX - x.shape[1] + 1):
        for i in range(gY - x.shape[0] + 1):
            h = np.zeros(gX*gY).reshape((gY,gX))
            for ii in range(x.shape[0]):
                for jj in range(x.shape[1]):
                    h[i+ii][j+jj] = x[ii][jj] 
            b.append(h)
    array.append(b)

In [633]:
array[1]

[array([[ 1.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]), array([[ 0.,  0.,  0.],
        [ 1.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]), array([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 1.,  0.,  0.],
        [ 0.,  0.,  0.]]), array([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 1.,  0.,  0.]]), array([[ 0.,  1.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]), array([[ 0.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]), array([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  0.]]), array([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  1.,  0.]]), array([[ 0.,  0.,  1.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]), array([[ 0.,  0.,  0.],
        [ 0.,  0.,  1.],
        [ 0.,  0.,  0.],
        [ 0.,  0

In [627]:
def solve(matrix, array, entry, draftSol ):
    if entry <= len(draftSol) - 1:
        global NumberOfSolution
        global solution 
        currentShapeFamily = array[entry]
        for i in range(len(currentShapeFamily)):
            if NumberOfSolution == 0:
                b = entry
                draftSol[entry] = i
                newShape = (matrix + currentShapeFamily[i])
                if NumberOfSolution == 0:
                    if ((newShape%2 == np.ones(gX*gY).reshape(gY,gX)).all()) & ( entry == len(draftSol)-1):
                        print newShape
                        NumberOfSolution = NumberOfSolution +1 
                        print "done"
                        solution =  draftSol

                    else:    
                        b = b + 1
                        solve(newShape, array, b , draftSol)
                        
                       

In [628]:
def summary(matrix, array):
    global NumberOfSolution
    global solution 
    NumberOfSolution = 0 
    entry = 0
    draftSol = -1*np.ones(len(array))
    solve(matrix, array, entry, draftSol ) 
    A = matrix 
    if solution is None:
        print 'fail'
    else:
        solution = solution.astype(int)
        print solution
        print "%%%%%%%%%%"
        print "operations: "
        for i in range(len(array)):
            print i+1
            print array[i][solution[i]] 
        print "%%%%%%%%%%"    
        print "step to result: "
        print A%2
        for i in range(len(array)):
            print i + 1
            A = A + array[i][solution[i]]
            print A%2

In [629]:
summary(matrix, array)

[[ 7.  7.  5.]
 [ 3.  5.  5.]
 [ 3.  5.  3.]
 [ 1.  1.  1.]]
done
[0 0 0 0 0 2 0 0 0 7 2]
%%%%%%%%%%
operations: 
1
[[ 1.  1.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
2
[[ 1.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
3
[[ 1.  0.  0.]
 [ 1.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
4
[[ 1.  1.  0.]
 [ 0.  1.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
5
[[ 1.  1.  1.]
 [ 1.  0.  1.]
 [ 1.  1.  1.]
 [ 0.  0.  0.]]
6
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 1.  1.  0.]
 [ 0.  0.  0.]]
7
[[ 0.  1.  1.]
 [ 1.  1.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
8
[[ 1.  1.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
9
[[ 1.  1.  0.]
 [ 0.  1.  1.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
10
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  1.  0.]]
11
[[ 0.  1.  1.]
 [ 0.  1.  1.]
 [ 0.  1.  1.]
 [ 0.  0.  0.]]
%%%%%%%%%%
step to result: 
[[ 0.  0.  1.]
 [ 0.  0.  0.]
 [ 1.  1.  1.]
 [ 1.  0.  1.]]
1
[[ 1.  1.  1.]
 [ 0.  0.  0.]
 [ 1.  1.  1.]
 [ 1.  0.  1.]]
2
[[ 0.  1.  1.]
 [ 0.  0.  0.]
 [ 1.  

In [488]:
matrix = np.zeros(gX*gY).reshape(gY,gX)
for i in range(gY):
    for j in range(gX):
        loc = Java.get_text().find('imgLocStr['+str(j)+']['+str(i) + ']')
        if Java.get_text()[loc+19:loc+22].encode('ascii','ignore') == "swo":
            matrix[i,j] = 1
matrix
        

array([[ 0.,  1.,  0.],
       [ 0.,  0.,  1.],
       [ 0.,  0.,  1.]])

In [555]:
int(Java.get_text()[pp.get_text().find('gY') + 6])

2

In [554]:
pp.get_text()[396:396+6]

u'gX = 3'

In [560]:
pp.get_text()[406:406+6]

u'gY = 3'

In [197]:
# 1 active shape
# 10 next shape,m
nextShape.find_all(cellspacing = "0")[0]

<table border="0" cellpadding="0" cellspacing="0"><tr><td height="10" width="10"></td><td><img border="0" height="10" src="http://images.neopets.com/medieval/shapeshifter/square.gif" width="10"/></td></tr><tr><td><img border="0" height="10" src="http://images.neopets.com/medieval/shapeshifter/square.gif" width="10"/></td><td><img border="0" height="10" src="http://images.neopets.com/medieval/shapeshifter/square.gif" width="10"/></td></tr></table>

In [190]:
len(p.find_all('td'))

4

In [191]:
ppp.find_all(cellspacing = "0")[0].find_all('tr')

[<tr><td height="10" width="10"></td><td><img border="0" height="10" src="http://images.neopets.com/medieval/shapeshifter/square.gif" width="10"/></td></tr>,
 <tr><td><img border="0" height="10" src="http://images.neopets.com/medieval/shapeshifter/square.gif" width="10"/></td><td><img border="0" height="10" src="http://images.neopets.com/medieval/shapeshifter/square.gif" width="10"/></td></tr>]

2
1
2
2
2
3
1
2
3
2


In [258]:
aaa[0][1]

array([[ 0.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  1.,  0.]])

In [199]:
import numpy as np
aa = []
for k in range(len(nextShape.find_all(cellspacing = "0")) + 1):
    if k == 0:
        q = activeShape
    elif k != 0:
        q = nextShape.find_all(cellspacing = "0")[k - 1]
    ii = len(q.find_all('tr'))
    jj = (len(q.find_all('td')) )/ii
    a = np.zeros(ii*jj).reshape(ii,jj)
    for i in range(ii):
        for j in range(jj):
            a[i][j] = len(q.find_all('td')[jj*i + j] )
    aa.append(a)
        

    

In [562]:
pp.get_text().find('gX = ')

396

In [64]:
pp.get_text()[401]

u'3'

In [65]:
pp.get_text().find('gY')

406

In [67]:
pp.get_text()[411]

u'3'

In [69]:
pp.get_text()[pp.get_text().find('gX') + 5]

u'3'

In [569]:
Java.get_text().find('gY = ')

514

In [575]:
r = 514
Java.get_text()[r:r+6]

u'gY = 4'

In [576]:
Java.get_text()[r+5]

u'4'

In [587]:
Java.get_text()

u'\n\n\n<!--\n\nfunction mouseAction(oM,xX,yY){\n\n  imgLocStr = new Array(4)\n  imgLocStr[0] = new Array(3)\n  imgLocStr[1] = new Array(3)\n  imgLocStr[2] = new Array(3)\n  imgLocStr[3] = new Array(3)\n  imgLocStr[0][0] = "swo"\n  imgLocStr[1][0] = "swo"\n  imgLocStr[2][0] = "gob"\n  imgLocStr[0][1] = "swo"\n  imgLocStr[1][1] = "gob"\n  imgLocStr[2][1] = "gob"\n  imgLocStr[0][2] = "swo"\n  imgLocStr[1][2] = "gob"\n  imgLocStr[2][2] = "gob"\n  imgLocStr[0][3] = "gob"\n  imgLocStr[1][3] = "gob"\n  imgLocStr[2][3] = "swo"\n\n  gX = 3;\n  gY = 4;\n\n  xX0 = xX+0;\n  xX1 = xX+1;\n  xX2 = xX+2;\n\n  yY0 = yY+0;\n  yY1 = yY+1;\n\n  if (xX0 < gX && yY0 < gY){ eval("document.i"+xX0+"_"+yY0+".src = \'http://images.neopets.com/medieval/shapeshifter/"+imgLocStr[xX0][yY0]+"_"+oM+".gif\'");}\n  if (xX1 < gX && yY0 < gY){ eval("document.i"+xX1+"_"+yY0+".src = \'http://images.neopets.com/medieval/shapeshifter/"+imgLocStr[xX1][yY0]+"_"+oM+".gif\'");}\n  if (xX1 < gX && yY1 < gY){ eval("document.i"+xX1